# modeling13-get-dataset-for-student-model

- Get dataset for student model

In [2]:
import numpy as np
import pathlib
from pprint import pprint
import pandas as pd
from util import utils
import re
import os

# Dev Prediction file
- Original FiD Dataset  
        Train  
        Dev   
        Test  
    

- Teacher Binray Classification Dataset
        Train   
        Dev -> Train/Dev   
        Test  

    
- Student Binary Classfication Dataset
        Train   
        Dev 
            Train
            Dev -> Train/Dev
        Test 

## Get Prediction of Teacher model on Dev
    - Get the gold model
    '/data/philhoon-relevance/binary-classification/results/NQ-DEV-DPR/5-fold/1/roberta-decisive_binary_gold_data_trial1/step_320/dev_prediction/prediction.json'

In [3]:
input_path = '/data/philhoon-relevance/binary-classification/results/NQ-DEV-DPR/5-fold/1/roberta-decisive_binary_gold_data_trial1/step_320/dev_prediction'
input_file = 'prediction.json'
input_file_path = os.path.join(input_path, input_file)
print(input_file_path)


/data/philhoon-relevance/binary-classification/results/NQ-DEV-DPR/5-fold/1/roberta-decisive_binary_gold_data_trial1/step_320/dev_prediction/prediction.json


In [4]:
prediction_data = utils.open_json(input_file_path)

In [5]:
len(prediction_data)

1704

In [6]:
prediction_data[0]

{'id': 0,
 'question': 'who sings does he love me with reba',
 'ctx': {'id': '11828866',
  'title': 'Does He Love You',
  'text': 'Does He Love You "Does He Love You" is a song written by Sandy Knox and Billy Stritch, and recorded as a duet by American country music artists Reba McEntire and Linda Davis. It was released in August 1993 as the first single from Reba\'s album "Greatest Hits Volume Two". It is one of country music\'s several songs about a love triangle. "Does He Love You" was written in 1982 by Billy Stritch. He recorded it with a trio in which he performed at the time, because he wanted a song that could be sung by the other two members'},
 'em': '1',
 'answers': ['Linda Davis'],
 'gold': 'Linda Davis',
 'inference': 'Linda Davis',
 'binary_inference': '1'}

## Check Original Data Size

In [ ]:
# binary_gold_dev = '/data/philhoon-relevance/binary-classification/NQ-DEV-DPR/5-fold/1/decisive_binary_gold_data/binary_decisive_gold_ctx100id_split_dev_1.json'
# binary_gold_train = '/data/philhoon-relevance/binary-classification/NQ-DEV-DPR/5-fold/1/decisive_binary_gold_data/binary_decisive_gold_ctx100id_split_train_1.json'

# binary_gold_dev = utils.open_json(binary_gold_dev)
# binary_gold_train = utils.open_json(binary_gold_train)


In [ ]:
# print(len(binary_gold_dev))
# print(len(binary_gold_train))

## Split by 5-fold

In [ ]:
from sklearn.model_selection import KFold
import random

In [ ]:
random.seed(42)

In [ ]:
print(len(prediction_data))

In [ ]:
kf = KFold(n_splits=5, random_state = 42, shuffle = True)

In [ ]:
kf.get_n_splits(prediction_data)

In [ ]:
output_path = '/data/philhoon-relevance/binary-classification/results/NQ-DEV-DPR/5-fold/1/roberta-decisive_binary_gold_data_trial1/step_320/dev_prediction'
output_directory = '5-fold'
output_path_directory = os.path.join(output_path, output_directory)
print(output_path_directory)

In [ ]:
os.makedirs(output_path_directory, exist_ok = True)

In [ ]:
for i, (train_index, test_index) in enumerate(kf.split(prediction_data)):
    output_path_i = output_path_directory + f'/{i+1}'
    os.makedirs(output_path_i, exist_ok = True)
    train_file_name = f'prediction_split_train_{i+1}.json'
    test_file_name = f'prediction_split_dev_{i+1}.json'
    
    train_path = os.path.join(output_path_i, train_file_name)
    test_path = os.path.join(output_path_i, test_file_name)
    
    print(f'train_path : {train_path}')
    print(f'dev_path : {test_path}')
    print(f"Fold {i}:")
    print(f"  Train: index={train_index[:10]}")
    print(f"  Test:  index={test_index[:10]}")
    train_split = [prediction_data[j] for j in train_index]
    test_split = [prediction_data[j] for j in test_index]
    print(f'size of train : {len(train_split)}')
    print(f'size of dev : {len(test_split)}')
    
    utils.save_json(train_split, train_path)
    utils.save_json(test_split, test_path)

# Get Train & Dev dataset fold1

In [7]:
train_path = '/data/philhoon-relevance/binary-classification/results/NQ-DEV-DPR/5-fold/1/roberta-decisive_binary_gold_data_trial1/step_320/dev_prediction/5-fold/1/prediction_split_train_1.json'
dev_path = '/data/philhoon-relevance/binary-classification/results/NQ-DEV-DPR/5-fold/1/roberta-decisive_binary_gold_data_trial1/step_320/dev_prediction/5-fold/1/prediction_split_dev_1.json'

In [8]:
train_data = utils.open_json(train_path)
dev_data = utils.open_json(dev_path)

In [9]:
len(train_data)

1363

In [10]:
len(dev_data)

341

In [30]:
train_data[15]

{'id': 65,
 'question': 'who plays caroline on the bold and beautiful',
 'ctx': {'id': '7171602',
  'title': 'Barbara Crampton',
  'text': '"Guiding Light" from 1993 to 1995 and left when her contract expired and when she got engaged to L.A.-based actor and director Kristoffer Tabori in April 1995. By September of the same year, their engagement was called off. In 1995, Crampton starred in "Castle Freak". From 1995 to 1998, Crampton portrayed Maggie Forrester on "The Bold and the Beautiful". In 1996, Crampton portrayed Carol in "Space Truckers". In 1997, Crampton guest starred on "The Nanny". The following year, she guest starred on "Party of Five" and starred in the film "The Godson". In 1999, Crampton guest starred on the television'},
 'em': '1',
 'answers': ['Linsey Godfrey'],
 'gold': 'Linsey Godfrey',
 'inference': 'Linsey Godfrey',
 'binary_inference': '0'}

In [31]:
cnt1_ = 0
cnt0_ = 0

for ins in dev_data:
    if ins['binary_inference'] == '1':
        cnt1_ += 1
    else:
        cnt0_ += 1
print(f'total instanes {len(train_data)}')
print(f'0 instanes {cnt0_}')
print(f'1 instanes {cnt1_}')

print(f'0 ratio {cnt0_/len(train_data)}')
print(f'1 instanes {cnt1_/len(train_data)}')

total instanes 1363
0 instanes 90
1 instanes 251
0 ratio 0.06603081438004402
1 instanes 0.18415260454878943


In [12]:
from src.data import BinaryCustomDatasetShuffle, BinaryCustomDatasetPredictionShuffle
from transformers import AutoTokenizer

In [13]:
tokenizer = AutoTokenizer.from_pretrained('roberta-large')

In [14]:
train_dataset = BinaryCustomDatasetPredictionShuffle(train_data, tokenizer, max_length = 200)

In [15]:
print(train_dataset[0])

{'input_ids': [0, 40018, 35, 54, 22707, 473, 37, 657, 162, 19, 5898, 102, 6, 1437, 1270, 35, 8901, 91, 3437, 370, 6, 1437, 5377, 4832, 8901, 91, 3437, 370, 22, 27847, 91, 3437, 370, 113, 16, 10, 2214, 1982, 30, 10234, 12996, 8, 7835, 312, 46313, 6, 8, 2673, 25, 10, 4279, 594, 30, 470, 247, 930, 3528, 1223, 3178, 1509, 30495, 1885, 8, 9369, 2505, 4, 85, 21, 703, 11, 830, 9095, 25, 5, 78, 881, 31, 1223, 3178, 18, 2642, 22, 19065, 990, 35865, 18071, 1596, 845, 85, 16, 65, 9, 247, 930, 18, 484, 3686, 59, 10, 657, 29884, 4, 22, 27847, 91, 3437, 370, 113, 21, 1982, 11, 12910, 30, 7835, 312, 46313, 4, 91, 2673, 24, 19, 10, 8566, 11, 61, 37, 3744, 23, 5, 86, 6, 142, 37, 770, 10, 2214, 14, 115, 28, 26115, 30, 5, 97, 80, 453, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [16]:
tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(train_dataset[0]['input_ids']))

'<s>question: who sings does he love me with reba,  title: Does He Love You,  context : Does He Love You "Does He Love You" is a song written by Sandy Knox and Billy Stritch, and recorded as a duet by American country music artists Reba McEntire and Linda Davis. It was released in August 1993 as the first single from Reba\'s album "Greatest Hits Volume Two". It is one of country music\'s several songs about a love triangle. "Does He Love You" was written in 1982 by Billy Stritch. He recorded it with a trio in which he performed at the time, because he wanted a song that could be sung by the other two members</s>'

## Get dataset for incremental Test
- from the output from modeling11-nli-binaryclassifier-modeling-from-scratch-prediction

## Incremental Testing Setting

### 3 Method (Check out README.md)

### Method 2 (2nd method of in README.md)
    - The sooner, The better approach
      - Keep the Positive Context in order
    - Since we testing it, let's keep sample size of 5
    - When there is no Exact Mathcing during the incremental inference e.g.) em_pattern = '00000'
        - Keep the whole context so that those cases will have False on Exact Match values
        
    - Patterns: 
        - first 1 : positive
        - first 0 : irrelevant 
            - we know that first is the positive context that includes answer
            - with that perspective, it is relevant in theory
            - ; however, dataset is created via BLEU score on two different corpus.
            - If FiD does not correctly infer the output, 
            - context what we concieved of artificial positive context is actually irrelevant to the query.
            - Also when realistic scenario, we don't know whether the first context contains the answer
        - 01 pattern : Positive 
        - 11 pattern : Strict Positive(relevant) or Naive Positive(positive)
        - 00 pattern : Strict Damaging(irrelevant) or Naive Damaging(damaging) 
        - 10 pattern : Damaging
        
        - 11 pattern : relevant vs positive
            Strict Positive
                Under strict rule : consider it as relevant ?(or irrelevant?)
                                in terms of strictly limiting the number of positive passages
            Naive Positive
                Under naive rule : consider it as positive
                                in that this would increase the number of positive passages
                
        - A00 pattern : irrelevant vs damaging
            if '1' does not occured in A, currnet passage is irrelevant
            if '1' occurred in A, current passage is damaging either irrelevant 
                Strict Damaging
                    Under strict rule : consider it as irrelevnat 
                                    in terms of strictly limiting the number of damaging passage
                Naive Damaging
                    Under naive rule : consider it as damaging 
                                    in that this would increase the number of damaging passages
        
    - Options1 : Removes only damaging
    - Options2 : Removes damaging + irrelevant
    - Options3 : Removes damaging + relevant
    - Options4 : Removes damaging + irrelevant + relevnat
        

# Input
    - result from 5-1
    - /data/philhoon-relevance/FiD/results/KILT_BM25_NQ/incremental_result/pos1_ctx5.json

# Trials
    - Need to check FiD input when there are less ctxs than n_context

# Binary Classification Output File

In [ ]:
sample_size = 100
input_file = f'/data/philhoon-relevance/binary-classification/results/NQ-DEV-DPR/5-fold/1/fid-encoder-adpative-decisive-trial7/step_360/intact_prediction/prediction.json'
input_data = utils.open_json(input_file)
pprint((input_data[5]))


In [ ]:
len(input_data)

In [ ]:
predict_id = {}
for ins in input_data:
    if ins['id'] not in predict_id:
        predict_id[ins['id']] = str(ins['binary_inference'])
    else:
        predict_id[ins['id']] += str(ins['binary_inference'])

In [ ]:
print(len(predict_id))

In [ ]:
print(len(predict_id[0]))

# Binary Classification Output File -> DPR format

In [ ]:
dpr_test_json = '/data/philhoon-relevance/binary-classification/NQ-TEST-DPR/ctx100id.json'

In [ ]:
dpr_test_data = utils.open_json(dpr_test_json)
print(dpr_test_data[0].keys())

In [ ]:
for ins in dpr_test_data:
    prediction_em_pattern = predict_id[ins['id']]
    ins['prediction_em_pattern'] = prediction_em_pattern
    

In [ ]:
print(dpr_test_data[1].keys())

In [ ]:
output_path = '/data/philhoon-relevance/binary-classification/results/NQ-DEV-DPR/5-fold/1/fid-encoder-adpative-decisive-trial7/step_360/intact_prediction'
output_file_name = 'ctx100id_prediction_em.json'
output_file_path = os.path.join(output_path, output_file_name)
print(output_file_path)

In [ ]:
utils.save_json(dpr_test_data, output_file_path)

# Method Selection

In [ ]:
def build_data_on_prediction(input_file, option, option_p, option_d, sample_size):
    '''
    input_file : incremental inference result from FiD from KILT-5-1
        path : /data/philhoon-relevance/FiD/results/KILT_BM25_NQ/incremental_result/pos1_ctx5.json
        
    output : FiD input json format
    
    option(required) : removing strategies
        op1 : removes damages only
        op2 : removes damaging + irrelevant
        op3 : removes damaging + relevant
        op4 : removes damaging + irrelevant + relevant
        
    option_p(required) : positive passage selection options
        strict : strict positive
            e.g.) 11 pattern 
                1st '1' is positive, 2nd '1' is relevant
        naive : naive positive
            e.g.) 11 pattern 
                1st '1' is positive, 2nd '1' is positive
                
    option_d(required) : damaging passage selection options
        strict : strict negative
            e.g.) A00 pattern 
                if there is at least one '1' occurred in A, 2nd '0' is irrelevant
        naive : naive damaging
            e.g.) A00 pattern 
                if there is at least one '1' occurred in A, 2nd '0' is damaging
    
    '''
    
    output_format = []
    null_em = '0' * sample_size
    
    # 'strict', 'naive'  
    # option_p = 'naive'
    # option_d = 'naive'
    # option = 'op4'

    for id_, instance in enumerate(input_file,1):
        template_dict = {}
        if 'id' in instance.keys():
            template_dict['id'] = instance['id']
        else:
            template_dict['id'] = str(id_)
        template_dict['answers'] = instance['answers']
        template_dict['question'] = instance['question']
#         template_dict['em_pattern'] = instance['em_pattern']
        template_dict['em_pattern'] = instance['em_pattern']
        template_dict['prediction_em_pattern'] = instance['prediction_em_pattern']

        em_pattern = instance['prediction_em_pattern']

        # when there is at least one EM in the accumulated inference
        if em_pattern != null_em:   
            new_ctx = []

            # relevant vs positive
            positve_ctx_lst = []
            relevant_ctx_lst = []

            # irrelevant vs damaging
            damaging_ctx_lst = []
            irrelevant_ctx_lst = []


            for idx_, ctx in enumerate(instance['ctxs']):

                # checking current em
                cur_em = em_pattern[idx_]
                pre_em_pattern = em_pattern[:idx_]


                # first 1 : positive
                if not pre_em_pattern and cur_em == '1':
                    positve_ctx_lst.append(ctx)

                # first 0 : irrelevant
                elif not pre_em_pattern and cur_em == '0':
                    irrelevant_ctx_lst.append(ctx)
                    
                # 01 pattern : positive 
                elif pre_em_pattern and pre_em_pattern[-1] == '0' and cur_em == '1':
                    positve_ctx_lst.append(ctx)

                # 10 pattern : damaging
                elif pre_em_pattern and pre_em_pattern[-1] == '1' and cur_em == '0':
                    damaging_ctx_lst.append(ctx)

                # 11 pattern : Strict Positive(relevant) or Naive Positive(positive)
                elif pre_em_pattern and pre_em_pattern[-1] == '1' and cur_em == '1':
                    if option_p == 'strict':
                        relevant_ctx_lst.append(ctx)

                    elif option_p == 'naive':
                        positve_ctx_lst.append(ctx)

                    else:
                        print('option_p should be either \'strict\' or \'naive\'')
                        return 

                # 00 pattern : Strict Damaging(irrelevant) or Naive Damaging(damaging) 
                elif pre_em_pattern and pre_em_pattern[-1] == '0' and cur_em == '0':
                    # if '1' does not occured in A, currnet passage is irrelevant
                    if not '1' in pre_em_pattern:
                        irrelevant_ctx_lst.append(ctx)

                    # if '1' occurred in A, 
                    else:
                        # strict : consider it as irrelevnat 
                        if option_d == 'strict':
                            irrelevant_ctx_lst.append(ctx)

                        # naive : consider it as damaging 
                        elif option_d == 'naive':
                            damaging_ctx_lst.append(ctx)

                        else:
                            print('option_p should be either \'strict\' or \'naive\'')
                            return 

            # op1 removes damages only
            if option == 'op1':
                new_ctx.extend(positve_ctx_lst)
                new_ctx.extend(relevant_ctx_lst)
                new_ctx.extend(irrelevant_ctx_lst)


            # op2 removes damaging + irrelevant
            elif option == 'op2':
                new_ctx.extend(positve_ctx_lst)
                new_ctx.extend(relevant_ctx_lst)

            # op3 : Removes damaging + relevant
            elif option == 'op3':
                new_ctx.extend(positve_ctx_lst)
                new_ctx.extend(irrelevant_ctx_lst)

            # op4 : Removes damaging + irrelevant + relevant
            elif option == 'op4':
                new_ctx.extend(positve_ctx_lst)

            else:
                print('option should be op1, op2, op3, op4')
                return 

            template_dict['ctxs'] = new_ctx
            output_format.append(template_dict)

        # when there is no EM in the accumulated inference
        else:
            template_dict['ctxs']= instance['ctxs']
            output_format.append(template_dict)
    
    print('==============instance finished======================')
    return output_format
        


In [ ]:
# option_dict = {
#     'op1' : 'remove_damage',
#     'op2' : 'remove_damage_irrelevant',
#     'op3' : 'remove_damage_relevant',
#     'op4' : 'remove_damage_irrelevant_relevant',
# }
# option = 'op4'

In [ ]:
# op1, op2 are duplicates
method_option_dict = {
    'op3' : 'remove_damage_relevant',
    'op4' : 'remove_damage_irrelevant_relevant',
}

In [ ]:
option_p_dict = {
    'strict' : 'strict_positive',
    'naive' : 'naive_positive',
}
# option_p = 'strict'

In [ ]:
option_d_dict = {
    'strict' : 'strict_damaging',
    'naive' : 'naive_damaging',
}
# option_d = 'strict'

In [ ]:
# input_file = utils.open_json('/data/philhoon-relevance/FiD/results/KILT_BM25_NQ/incremental_result/pos1_ctx5.json')
# output_path = '/data/philhoon-relevance/FiD/open_domain_data/NQ_KILT_BM25_SELECTION'

# TEST set Inference

In [ ]:
input_file = '/data/philhoon-relevance/binary-classification/results/NQ-DEV-DPR/5-fold/1/fid-encoder-adpative-decisive-trial7/step_360/intact_prediction/ctx100id_prediction_em.json'
input_ = utils.open_json(input_file)
len(input_)

In [ ]:
output_path = '/data/philhoon-relevance/binary-classification/results/NQ-DEV-DPR/5-fold/1/fid-encoder-adpative-decisive-trial7/step_360/intact_prediction/selection_methods'

print(output_path)


In [ ]:
os.makedirs(output_path, exist_ok = True)

In [ ]:
output_path

In [ ]:
for o_ in method_option_dict.keys():
    for op in option_p_dict.keys():
        for od in option_d_dict.keys():
            option = o_
            option_p = op
            option_d = od
            
            if option == 'op4' and option_p == 'strict' and option_d == 'strict':
                continue
            if option == 'op4' and option_p == 'naive' and option_d == 'strict':
                continue 
            
            filename = f'{option_p_dict[option_p]}_{option_d_dict[option_d]}_{method_option_dict[option]}.json'

            output_file = os.path.join(output_path, filename)
            output_format = build_data_on_prediction(input_, option, option_p, option_d, sample_size)
            
            utils.save_json(output_format, output_file)
            print(f'{filename} save on \n {output_path}')
#             print(f'option : {option}')
#             print(f'option_p : {option_p}')
#             print(f'option_d : {option_d}')
#             print(f'filename : {filename}')
#             print(f'output_file : {output_file}')
            

- Method3. '00011011000'  -> 
<span style="color:blue">0</span><span style="color:blue">0</span><span style="color:blue">0</span><span style="color:red">1</span></span><span style="color:red">1</span>0<span style="color:red">1</span></span><span style="color:red">1</span>000  (__0.73651__)     

- Method4. '00011011000'  -> 
<span style="color:blue">0</span><span style="color:blue">0</span><span style="color:blue">0</span><span style="color:red">1</span>10<span style="color:red">1</span>1000 (_0.73611_)    

- Method1. '00011011000'  -> 000<span style="color:red">1</span><span style="color:red">1</span>0<span style="color:red">1</span><span style="color:red">1</span>000 (0.71130)    

- Method2. '00011011000'  -> 000<span style="color:red">1</span>10<span style="color:red">1</span>1000  (0.70972)        
- Method6. '00011011000'  -> <span style="color:blue">0</span><span style="color:blue">0</span><span style="color:blue">0</span><span style="color:red">1</span>10<span style="color:red">1</span>10<span style="color:blue">0</span><span style="color:blue">0</span> (0.68609)    

- Method5. '00011011000'  -> <span style="color:blue">0</span><span style="color:blue">0</span><span style="color:blue">0</span><span style="color:red">1</span><span style="color:red">1</span>0<span style="color:red">1</span></span><span style="color:red">1</span>0<span style="color:blue">0</span><span style="color:blue">0</span>    (0.68452) 

In [ ]:
method_dict = {'strict_positive_strict_damaging_remove_damage_relevant.json' : 'Method6', 
'strict_positive_naive_damaging_remove_damage_relevant.json' : 'Method4', 
'naive_positive_strict_damaging_remove_damage_relevant.json' : 'Method5', 
'naive_positive_naive_damaging_remove_damage_relevant.json' : 'Method3', 
'strict_positive_naive_damaging_remove_damage_irrelevant_relevant.json' : 'Method2', 
'naive_positive_naive_damaging_remove_damage_irrelevant_relevant.json' : 'Method1',
}

In [ ]:
test_path= '/data/philhoon-relevance/binary-classification/results/NQ-DEV-DPR/5-fold/1/roberta-decisive_binary_gold_data_trial1/step_320/intact_prediction/selection_methods'
# method 2
test_file = 'strict_positive_naive_damaging_remove_damage_irrelevant_relevant.json'
test_file_path = os.path.join(test_path, test_file)

In [ ]:
test_data = utils.open_json(test_file_path)

In [ ]:
len(test_data)

In [ ]:
test_data[0].keys()

In [ ]:
for instance in test_data:
    print(instance['em_pattern'])
    print(instance['prediction_em_pattern'])
    print(len(instance['ctxs']))
    print('----')

## NQ dev set

In [ ]:
sample_size = 100
input_file = f'/data/philhoon-relevance/FiD/results/NQ_DPR/DEV/incremental_result_{sample_size}/ctx{sample_size}.json'
output_path = f'/data/philhoon-relevance/FiD/open_domain_data/NQ_DPR_DEV_SELECTION/ctx_{sample_size}'

In [ ]:
input_ = utils.open_json(input_file)

In [ ]:
output_path

In [ ]:
len(input_)

In [ ]:
for o_ in method_option_dict.keys():
    for op in option_p_dict.keys():
        for od in option_d_dict.keys():
            option = o_
            option_p = op
            option_d = od
            
            if option == 'op4' and option_p == 'strict' and option_d == 'strict':
                continue
            if option == 'op4' and option_p == 'naive' and option_d == 'strict':
                continue 
            
            filename = f'{option_p_dict[option_p]}_{option_d_dict[option_d]}_{method_option_dict[option]}.json'

            output_file = os.path.join(output_path, filename)
            output_format = build_data(input_, option, option_p, option_d, sample_size)
            
            utils.save_json(output_format, output_file)
            print(f'{filename} save on \n {output_path}')
#             print(f'option : {option}')
#             print(f'option_p : {option_p}')
#             print(f'option_d : {option_d}')
#             print(f'filename : {filename}')
#             print(f'output_file : {output_file}')
            

# Method explanation
- Redundancies in selection strategies -> remove them -> 6

#### Method1. Include passages that corresponds to 1s
- all __option2__ belong to this case
- option4_naive_naive, option4_naive_strict

'00011011000'  -> 000<span style="color:red">1</span><span style="color:red">1</span>0<span style="color:red">1</span><span style="color:red">1</span>000  
- red 1s

#### Method2. Include passages that corresponds to first 1s
- option4_strict_naive, option4_strict_strcit 

'00011011000'  -> 
000<span style="color:red">1</span>10<span style="color:red">1</span>1000  
- red 1s
- removing consecutive passages when the previous output is correct as well

#### Method3. Include passages that corresponds to 1s + Included First appeared consecutive 0s
- option1_naive_naive, option1_strict_naive
- option3_naive_naive

'00011011000'  -> 
<span style="color:blue">0</span><span style="color:blue">0</span><span style="color:blue">0</span><span style="color:red">1</span></span><span style="color:red">1</span>0<span style="color:red">1</span></span><span style="color:red">1</span>000  
- red 1s and blue 0s
- usually top-retrieved results contain the answer 

#### Method4. Include passages that corresponds to first 1s + Included First appeared consecutive 0s
- option3_strict_naive

'00011011000'  -> 
<span style="color:blue">0</span><span style="color:blue">0</span><span style="color:blue">0</span><span style="color:red">1</span>10<span style="color:red">1</span>1000  
- red 1s and blue 0s
- removing consecutive passages when the previous output is correct as well
- usually top-retrieved results contain the answer 

#### Method5. Include passages that corresponds to 1s + Remove only damaging
- option1_naive_strict, option1_strict_strict
- option3_naive_strict

'00011011000'  -> 
<span style="color:blue">0</span><span style="color:blue">0</span><span style="color:blue">0</span><span style="color:red">1</span></span><span style="color:red">1</span>0<span style="color:red">1</span></span><span style="color:red">1</span>0<span style="color:blue">0</span><span style="color:blue"><span style="color:blue">0</span><span style="color:blue"> 
- red 1s and blue 0s
- only removes damaging passages for comparison with Method3, Method4
- usually top-retrieved results contain the answer 

#### Method6. Include passages that corresponds to first 1s  + Remove only damaging
- option3_strict_strict

'00011011000'  -> 
<span style="color:blue">0</span><span style="color:blue">0</span><span style="color:blue">0</span><span style="color:red">1</span>10<span style="color:red">1</span>10<span style="color:blue">0</span><span style="color:blue">0</span>
- red 1s and blue 0s
- removing consecutive passages when the previous output is correct as well
- only removes damaging passages for comparison with Method3, Method4
- usually top-retrieved results contain the answer 

## Conclusion
- option1, option2 are not needed
- Method1 : option4_naive_naive
- Method2 : option4_strict_naive 
- Method3 : option3_naive_naive
- Method4 : option3_strict_naive
- Method5 : option3_naive_strict
- Method6 : option3_strict_strict